In [ ]:
!pip install -q openai tqdm

import pandas as pd
import requests
import time
from tqdm import tqdm

In [ ]:
# Modelos a testar (via OpenRouter)
modelos = [
    "qwen/qwen-2.5-7b-instruct"  
    #"mistralai/mistral-7b-instruct",
    #"google/gemma-3-4b-it"
   # "meta-llama/llama-3.1-8b-instruct",
]

# Estratégias de mitigação via prompt
prompts_mitigacao = {
    "aware": (
        "When analyzing information, remain aware of confirmation bias — "
        "the tendency to favor data that supports prior beliefs while ignoring conflicting evidence. "
        "Actively seek out contradictory information and incorporate it into your reasoning. "
        "Strive for a balanced and evidence-based response."
        "Then give your answer in a direct way."
    ),
     "autocritica_simulada": (
        "Answer the question first. Then, reflect on your reasoning: "
        "Was any key evidence overlooked? Could your response have been influenced by confirmation bias? "
        "Revise your answer and state your impressions in an direct way."
    ),
    "multiplas_fontes": (
        "Ensure your answer is informed by diverse sources, not just those that align with initial assumptions. "
        "Actively seek out credible perspectives that challenge your viewpoint to increase objectivity and accuracy."
        "Give your answer in a direct way."
    ),
    "protocolo_verificacao": (
        "Before forming a conclusion, apply this verification protocol:\n"
        "(1) Search for disconfirming evidence.\n"
        "(2) Consider the strongest counterarguments.\n"
        "(3) Check for cherry-picked data.\n"
        "(4) Review sources that challenge your assumptions.\n"
        "(5) Identify what evidence would change your mind.\n"
        "(6) Give your answer in a direct way .\n"

    )

}


In [ ]:
def montar_prompt(instrucao, pergunta):
    return f"{instrucao}\n\nQuestion: {pergunta}"


In [ ]:
def consultar_openrouter(prompt, chave_api, model):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {chave_api}",
        "HTTP-Referer": "https://chat.openai.com/",
        "X-Title": "Viés Prompt Mitigation"
    }

    body = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']


In [ ]:
def testar_mitigacao_prompt(arquivo_perguntas, chave_api):
    perguntas_df = pd.read_csv(arquivo_perguntas)

    for modelo in tqdm(modelos, desc="Modelos"):
        resultados = []

        for _, linha in tqdm(perguntas_df.iterrows(), total=perguntas_df.shape[0], desc="Perguntas", leave=False):
            pergunta = linha["pergunta"]
            pergunta_id = linha["id"]

            linha_resultado = {
                "id": pergunta_id,
                "pergunta": pergunta,
                "modelo": modelo,
                "resposta_aware": "",
                "resposta_autocritica_simulada": "",
                "resposta_multiplas_fontes": "",
                 "resposta_protocolo_verificacao": ""
            }

            for chave_prompt, instrucao in prompts_mitigacao.items():
                prompt_final = montar_prompt(instrucao, pergunta)

                try:
                    resposta = consultar_openrouter(prompt_final, chave_api, model=modelo)
                    time.sleep(20)  # evita erro 429
                except Exception as e:
                    resposta = f"Erro: {str(e)}"

                linha_resultado[f"resposta_{chave_prompt}"] = resposta

            resultados.append(linha_resultado)

        df_resultado = pd.DataFrame(resultados)
        nome_arquivo = f"respostas_prompt_mitigacao_{modelo.replace('/', '_').replace(':', '-')}.csv"
        df_resultado.to_csv(nome_arquivo, index=False)
        print(f"✅ Resultados salvos em: {nome_arquivo}")


In [ ]:
# 🔑 Sua chave da OpenRouter (coloque a sua aqui)
chave_api = ""  # substitua pela sua chave real

# 🏁 Executar o pipeline com seu arquivo
testar_mitigacao_prompt("perguntas_enviesadas.csv", chave_api)


Modelos: 100%|██████████| 1/1 [1:22:35<00:00, 4955.57s/it]

✅ Resultados salvos em: respostas_prompt_mitigacao_qwen_qwen-2.5-7b-instruct.csv
